In [1]:
import os
os.chdir('..')

In [6]:
!pip install tqdm

     |████████████████████████████████| 75 kB 5.5 MB/s eta 0:00:011


In [2]:
import random

from tqdm import tqdm
from collections import Counter

In [3]:
graph = list()

with open('./com-dblp/com-dblp-ungraph-100.txt', 'r') as f:
    for line in f.readlines():
        line = line.strip().split(' ')
        n1 = int(line[0])
        n2 = int(line[1])
        graph.append((n1, n2))
        
labels = list()
with open('./com-dblp/com-dblp.all.cmty.update.txt', 'r') as f:
    for line in f.readlines():
        line = line.strip().split(' ')
        n = int(line[0])
        i = int(line[1])
        labels.append((n, i))

In [76]:
def drop_graph(graph, labels, rate=0.5):
    new_graph = list()
    nodes = set()
    for n1, n2 in graph:
        nodes.add(n1)
        nodes.add(n2)
    nodes = list(nodes)
    nodes = random.choices(nodes, k=int(len(nodes)*rate))
    nodes = set(nodes)
    
    for n1, n2 in graph:
        if n1 in nodes and n2 in nodes:
            new_graph.append((n1, n2))
    
    
    new_labels = list()
    for n, i in labels:
        if n in nodes:
            new_labels.append((n, i)) 
    
    return new_graph, new_labels

In [97]:
raw_cluster_id = [i for _, i in labels]

In [98]:
cluster_counter = Counter(raw_cluster_id)

In [46]:
def drop_graph2(graph, labels, max_=100):
    raw_cluster_id = [i for _, i in labels]
    cluster_counter = Counter(raw_cluster_id)
    cluster_list = [(id_, count) for id_, count in cluster_counter.items()]
    ids = sorted([(id_, count) for id_, count in cluster_counter.items()], key=lambda x: x[1], reverse=True)
    keep_ids = set([id_ for id_, _ in ids[:max_]])
    
    new_labels = list()
    new_graph = list()
    nodes = set()
    
    for n, i in labels:
        if i in keep_ids:
            new_labels.append((n, i))
            nodes.add(n)
    
    for n1, n2 in graph:
        if n1 in nodes or n2 in nodes:
            new_graph.append((n1, n2))
    
    return new_graph, new_labels

In [49]:
new_graph, new_labels = drop_graph2(graph, labels, max_=2)
len(new_graph), len(new_labels)

(107915, 13509)

In [50]:
with open('./com-dblp/com-dblp-ungraph-custom.txt', 'w+') as f:
    for line in new_graph:
        f.write(f'{line[0]} {line[1]}' + '\n')
        
with open('./com-dblp/com-dblp-label-custom.txt', 'w+') as f:
    for line in new_labels:
        f.write(f'{line[0]} {line[1]}' + '\n')